In [1]:
from tqdm import tqdm, trange  # Use this for scripts outside notebooks


In [2]:
import pandas as pd
file_path = 'D:\vidhya\vidhya.csv'
df = pd.read_csv('vidhya.csv')
df.head()

,course-card href,course-card__body,course-card__body 2,course-card__lesson-count,course-card__price,review__stars-count
0,https://courses.analyticsvidhya.com/courses/ge...,All Courses,GenAI Applied to Quantitative Finance: For Con...,5 Lessons,Free,NaN
1,https://courses.analyticsvidhya.com/courses/na...,All Courses,Navigating LLM Tradeoffs: Techniques for Speed...,6 Lessons,Free,NaN
2,https://courses.analyticsvidhya.com/courses/cr...,"All Courses, \n \n \n \n ...",Creating Problem-Solving Agents using GenAI fo...,6 Lessons,Free,NaN
3,https://courses.analyticsvidhya.com/courses/im...,"All Courses, \n \n \n \n ...",Improving Real World RAG Systems: Key Challeng...,12 Lessons,Free,NaN
4,https://courses.analyticsvidhya.com/courses/ch...,"All Courses, \n \n \n \n ...",Framework to Choose the Right LLM for your Bus...,6 Lessons,Free,NaN


In [3]:
# Clean and prepare the dataset by extracting course titles, descriptions, and links
# Removing unnecessary spaces and line breaks from course-card__body 2 for better readability
df_clean = pd.DataFrame({
    'course_link': df['course-card href'],
    'course_title': df['course-card__body 2'].str.strip(),
    'lessons_count': df['course-card__lesson-count'].str.strip(),
    'price': df['course-card__price'].str.strip()
})

# Remove any unwanted characters from course titles
df_clean['course_title'] = df_clean['course_title'].str.replace(r'\n\s+', '', regex=True).str.strip()

# Display cleaned dataset
df_clean.head()


,course_link,course_title,lessons_count,price
0,https://courses.analyticsvidhya.com/courses/ge...,GenAI Applied to Quantitative Finance: For Con...,5 Lessons,Free
1,https://courses.analyticsvidhya.com/courses/na...,Navigating LLM Tradeoffs: Techniques for Speed...,6 Lessons,Free
2,https://courses.analyticsvidhya.com/courses/cr...,Creating Problem-Solving Agents using GenAI fo...,6 Lessons,Free
3,https://courses.analyticsvidhya.com/courses/im...,Improving Real World RAG Systems: Key Challeng...,12 Lessons,Free
4,https://courses.analyticsvidhya.com/courses/ch...,Framework to Choose the Right LLM for your Bus...,6 Lessons,Free


In [4]:
from sentence_transformers import SentenceTransformer

# Initialize the pre-trained model for generating embeddings (e.g., Sentence-BERT)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for course titles/descriptions
course_embeddings = model.encode(df_clean['course_title'].tolist(), show_progress_bar=True)

# Add embeddings back to the dataframe
df_clean['embeddings'] = course_embeddings.tolist()


c:\Users\ABHISHEK\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
from sentence_transformers import SentenceTransformer

# Initialize the pre-trained model for generating embeddings (using a model like Sentence-BERT)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for the course titles and descriptions
course_embeddings = model.encode(df_clean['course_title'].tolist(), show_progress_bar=True)

# Save the embeddings and the cleaned data for further use
df_clean['embeddings'] = course_embeddings.tolist()

# Display the cleaned dataset with embeddings
df_clean.head()


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

,course_link,course_title,lessons_count,price,embeddings
0,https://courses.analyticsvidhya.com/courses/ge...,GenAI Applied to Quantitative Finance: For Con...,5 Lessons,Free,"[-0.05822283402085304, 0.053621310740709305, -..."
1,https://courses.analyticsvidhya.com/courses/na...,Navigating LLM Tradeoffs: Techniques for Speed...,6 Lessons,Free,"[0.07394049316644669, -0.05113427713513374, -0..."
2,https://courses.analyticsvidhya.com/courses/cr...,Creating Problem-Solving Agents using GenAI fo...,6 Lessons,Free,"[-0.06169634684920311, -0.015118584036827087, ..."
3,https://courses.analyticsvidhya.com/courses/im...,Improving Real World RAG Systems: Key Challeng...,12 Lessons,Free,"[-0.07903501391410828, 0.0515003576874733, 0.0..."
4,https://courses.analyticsvidhya.com/courses/ch...,Framework to Choose the Right LLM for your Bus...,6 Lessons,Free,"[0.061196036636829376, -0.04366641491651535, -..."


In [6]:
df_clean.to_csv('cleaned_courses.csv', index=False)


In [7]:
import gradio as gr
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Load the cleaned data with embeddings
df_clean = pd.read_csv('cleaned_courses.csv')  # Assuming you save the cleaned data
df_clean['embeddings'] = df_clean['embeddings'].apply(eval)  # Convert from string to list

# Load the same model used to generate embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to search for the most relevant courses
def search_courses(query):
    query_embedding = model.encode([query])[0]
    
    # Compute cosine similarity between query and course embeddings
    similarities = cosine_similarity([query_embedding], df_clean['embeddings'].tolist())[0]
    
    # Sort results based on similarity score
    df_clean['similarity'] = similarities
    top_courses = df_clean.sort_values(by='similarity', ascending=False).head(5)
    
    # Format the results
    results = []
    for _, row in top_courses.iterrows():
        course_info = f"**Title**: {row['course_title']}\n**Link**: {row['course_link']}\n**Lessons**: {row['lessons_count']}\n**Price**: {row['price']}\n"
        results.append(course_info)
    
    return "\n\n".join(results)

# Create the Gradio interface
# Create the Gradio interface
iface = gr.Interface(fn=search_courses, 
                     inputs="text", 
                     outputs="markdown",
                     title="Smart Course Search Tool",
                     description="Enter your query to search for relevant courses")

# Launch the interface with share=True to create a public link
iface.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://05d3a2a2e12f64e344.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [8]:
pip install gradio


In [9]:
df['course-card__body 2'][3]

'Improving Real World RAG Systems: Key Challenges & Practical Solutions'